In [34]:
import pandas as pd


In [35]:
df = pd.read_csv("cancer.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [36]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [37]:
df["diagnosis"] = df["diagnosis"].apply(lambda x: 1 if x == "M" else 0)

In [38]:
X = df[['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']]

y = df["diagnosis"]

In [39]:
X.shape

(569, 30)

In [40]:
y.shape

(569,)

In [41]:
# dropping the null values from X

X = X.dropna(axis = 1)

In [42]:
y.value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

#### HoldOut Validation Approach - Train Test split

In [44]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 100)

model = DecisionTreeClassifier()
model.fit(X_train,y_train)
acc = model.score(X_test, y_test)
print(acc)

0.9415204678362573


Disadvantages:
- Single type of train test we are doing, if we cahnge the random state we get differnet type of accuracies

#### K fold cross validation

Suppose i want to do 10 splits, so 10 splits of train and test data will be taken, ml model will be applied and accuracy will be calculated for all 10 subsets. Finally the average of all this 10 accuracies will be told to the stake holder

In [45]:
from sklearn.model_selection import KFold
model = DecisionTreeClassifier()
kfold_validation = KFold(10) # 10 different splits is done and here we get it

In [46]:
import numpy as np
from sklearn.model_selection import cross_val_score

# cross val score shows our accuracies of 10 splits

results = cross_val_score(model,X,y,cv = kfold_validation)
print(results)
print(np.mean(results))

[0.9122807  0.9122807  0.89473684 0.94736842 0.9122807  0.96491228
 0.89473684 0.96491228 0.98245614 0.89285714]
0.9278822055137844


#### Stratified K-Fold Cross Validation

If dataset is imbalanced, correct proportions of our category may not be taken. If one category is in 100s and other has 900 values then the ratio is 1:9, so the proportions may not be taken correctly while splitting train test. So stratified k fold cross validation helps in taking the correct proprotion of our train test features

In [48]:
# we us eit when our dataset is imbalanced

from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits=5)
model = DecisionTreeClassifier()
scores = cross_val_score(model,X,y,cv=skfold)
print(np.mean(scores))

0.9191429902189101


#### Leave One Out Cross Validation(LOOCV)

From the entire dataset we take one data point randomely and all remaining records will be my training dataset

In [49]:
from sklearn.model_selection import LeaveOneOut
model = DecisionTreeClassifier()
leave_validation = LeaveOneOut()
results = cross_val_score(model, X, y, cv = leave_validation)

In [50]:
print(np.mean(results))

0.9314586994727593


#### Repeated Random Train-Test Splits

This technique is a hybrid of traditional train-test splitting and the k-fold cross validation method

In [51]:
from sklearn.model_selection import ShuffleSplit
model = DecisionTreeClassifier()
ssplit = ShuffleSplit(n_splits=10, test_size = 0.3)
results = cross_val_score(model, X, y, cv=ssplit)

In [52]:
results

array([0.94152047, 0.9122807 , 0.91812865, 0.90643275, 0.90643275,
       0.9122807 , 0.92982456, 0.9005848 , 0.95906433, 0.91812865])

In [53]:
np.mean(results)

0.9204678362573098